In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader


In [3]:
model = models.vgg13(weights=models.VGG13_Weights.DEFAULT)

for param in model.features.parameters():
    param.requires_grad = False

num_features = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)  
)

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [8]:
train_dataset = datasets.ImageFolder(root='dataset/train', transform=train_transform)
val_dataset = datasets.ImageFolder(root='dataset/val', transform=test_val_transform)
test_dataset = datasets.ImageFolder(root='dataset/test', transform=test_val_transform)

trainloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=100, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=100, shuffle=False)


In [7]:
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
def train(model, trainloader, optimizer, criterion, device):
    model.train()
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    return loss.item()

def validate(model, validationloader, device):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for data in validationloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

for epoch in range(20):  # Number of epochs
    train_loss = train(model, trainloader, optimizer, criterion, device)
    validation_accuracy = validate(model, valloader, device)
    print(f'Epoch {epoch+1}, Loss: {train_loss}, Validation Accuracy: {validation_accuracy}')


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.56 GiB. GPU 0 has a total capacty of 3.81 GiB of which 694.19 MiB is free. Including non-PyTorch memory, this process has 2.24 GiB memory in use. Of the allocated memory 2.12 GiB is allocated by PyTorch, and 22.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to('cuda')
        labels = labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total}%')


Accuracy of the network on the test images: 96%
